In [1]:
from pynq import Overlay, GPIO, Register, allocate, MMIO
import os
import struct
import numpy as np

def float_to_int(f):
    return (struct.unpack('<I', struct.pack('<f', f))[0])

def int_to_float(i):
    return (struct.unpack('<f', struct.pack('<I', i))[0])
    


In [2]:
overlay = Overlay("nn.bit")
fcc1=overlay.forward_fcc_0
fcc1.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=0),
  w = Register(w=0),
  y = Register(y=0),
  b = Register(b=0),
  xdimension = Register(xdimension=0),
  ydimension = Register(ydimension=0)
}

In [3]:
BASE_ADDRESS_X=0x4001_0000
BASE_ADDRESS_W=0x4001_0100
BASE_ADDRESS_Y=0x4001_0200
BASE_ADDRESS_B=0x4001_0300

ADDRESS_RANGE=0xF0

mmio_x= MMIO(BASE_ADDRESS_X,ADDRESS_RANGE)
mmio_w= MMIO(BASE_ADDRESS_W,ADDRESS_RANGE)
mmio_y= MMIO(BASE_ADDRESS_Y,ADDRESS_RANGE)
mmio_b= MMIO(BASE_ADDRESS_B,ADDRESS_RANGE)

OFFSET_X=0x0
OFFSET_Y=0x0
OFFSET_W=0x0
OFFSET_B=0x0

xdim=3
ydim=4

In [4]:
for i in range(xdim):
    mmio_x.write(OFFSET_X,float_to_int(i*0.5))    
    OFFSET_X=OFFSET_X+4

for i in range(ydim):
    for j in range(xdim):
        mmio_w.write(OFFSET_W,float_to_int((i+1)*0.3))
        OFFSET_W += 4

for i in range(ydim):
    mmio_y.write(OFFSET_Y,float_to_int(0))
    mmio_b.write(OFFSET_B,float_to_int(2.5))
    OFFSET_Y=OFFSET_Y+4
    OFFSET_B=OFFSET_B+4

In [5]:


fcc1.register_map.x=BASE_ADDRESS_X
fcc1.register_map.y=BASE_ADDRESS_Y
fcc1.register_map.w=BASE_ADDRESS_W
fcc1.register_map.b=BASE_ADDRESS_B
fcc1.register_map.xdimension=xdim
fcc1.register_map.ydimension=ydim

In [6]:
fcc1.write(0x00, 1)
fpga_state = fcc1.read(0x00)

max_try = 1000000
while fpga_state != 6 and fpga_state != 4:
    fpga_state = fcc1.read(0x00)
    max_try = max_try -1
    if max_try == 0:
        print("ERROR: Can't go ahead")
        fcc1.write(0x00, 4)
        break

fcc1.write(0x00, 4)
# print(xbuffer)
# print(wbuffer)
# print(ybuffer)
# y[:ydim] = ybuffer[:ydim]

In [7]:
y=[]
OFFSET_X=0x0
OFFSET_Y=0x0
OFFSET_W=0x0
OFFSET_B=0x0
for i in range(ydim):
    y.append(int_to_float(mmio_y.read(OFFSET_Y)))
    OFFSET_Y= OFFSET_Y+4

    
print(y)

[2.950000047683716, 3.4000000953674316, 3.8499999046325684, 4.300000190734863]


0
